In [52]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import pairwise_distances

In [50]:
data = pd.read_csv("raw transcripts/this_american_life.csv")
titles = pd.read_csv("raw data/episode_info_clean.csv")

data.sort_values("episode")
titles.sort_values("episode_number", inplace=True)

df = data.merge(titles, left_on="episode", right_on="episode_number").drop(columns=['episode', "episode_number", "radio_date"])

df.set_index("title", inplace = True)

df.text = df.text.str.lower()
for p in ".,?$%&^*!":
    df.text = df.text.str.replace(p, "")

In [37]:
# fastest source of contractions I could find: https://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python
contractions = { 
"ain't": "am not / are not / is not / has not / have not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is / how does",
"I'd": "I had / I would",
"I'd've": "I would have",
"I'll": "I shall / I will",
"I'll've": "I shall have / I will have",
"I'm": "I am",
"I've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}

In [51]:
for k, v in contractions.items():
    df.text = df.text.str.replace(k.lower(), v.lower())

In [45]:
df.head()

,text
title,
New Beginnings,joe franklin i am ready it has / it is ira gl...
Small Scale Sin,ok three boys aged 13 15 and 16 all three cho...
Poultry Slam 1995,in danielle's house ever since she was a girl...
Vacations,the thing about hawaii is that before you go ...
Anger and Forgiveness,hi it has / it is ira glass recording this i...


In [54]:
tvec = TfidfVectorizer(stop_words="english", max_features=600)
port = PorterStemmer()
stops = stopwords.words("english")

In [57]:
for i, episode in enumerate(df.text):
    df.text[i] = " ".join([port.stem(str(s)) for s in str(df.text[i]).split(" ") if not str(s) in stops])

In [71]:
recommender = pairwise_distances(tvec.fit_transform(df.text), metric="cosine")

In [73]:
rec_df = pd.DataFrame(recommender, index=df.index, columns=df.index)

In [74]:
rec_df.head()

title,New Beginnings,Small Scale Sin,Poultry Slam 1995,Vacations,Anger and Forgiveness,Christmas,Quitting,New Year,Julia Sweeney,Double Lives,...,Who You Gonna Call?,We Are in the Future,Private Geography,Essay B,White Haze,Suitable for Children,In the Shadow of the City 2017,Expect Delays,Things I Mean to Know,So a Monkey and a Horse Walk Into a Bar
title,,,,,,,,,,,,,,,,,,,,,
New Beginnings,0.000000,0.324579,0.456044,0.401299,0.308174,0.496257,0.544256,0.352169,0.370711,0.458901,...,0.338523,0.376144,0.480824,0.460404,0.426626,0.384460,0.395779,0.349688,0.378673,0.508636
Small Scale Sin,0.324579,0.000000,0.496025,0.443647,0.374389,0.549141,0.589104,0.379815,0.386164,0.500092,...,0.317098,0.356763,0.395483,0.397551,0.345630,0.361571,0.309859,0.375309,0.348119,0.517147
Poultry Slam 1995,0.456044,0.496025,0.000000,0.516213,0.500610,0.618920,0.661467,0.511914,0.501031,0.564402,...,0.485591,0.549179,0.588773,0.573658,0.579979,0.546698,0.531092,0.535478,0.534279,0.636094
Vacations,0.401299,0.443647,0.516213,0.000000,0.432085,0.520098,0.583485,0.461557,0.468572,0.439924,...,0.480433,0.429344,0.505546,0.497360,0.508659,0.397312,0.464805,0.356654,0.472730,0.529896
Anger and Forgiveness,0.308174,0.374389,0.500610,0.432085,0.000000,0.495060,0.558426,0.334720,0.446506,0.434194,...,0.373133,0.410908,0.470980,0.450222,0.418573,0.395363,0.440034,0.324421,0.409648,0.522242


In [95]:
def recommend(episode, top=11):
    find = rec_df.loc[rec_df.index.str.contains(episode)].index[0]
    return pd.DataFrame(rec_df[find].sort_values()[1:top])

In [96]:
recommend("Bad")

,How Bad Is Bad?
title,
Small Scale Sin,0.082818
Suckers,0.290531
Hoaxing Yourself,0.291568
Call For Help,0.293973
I Thought I Knew You,0.302260
This I Used to Believe,0.302327
Nice Work If You Can Get It,0.303125
How to Take Money from Strangers,0.303604
What I Should’ve Said,0.304812
